In [1]:
import pickle

with open('mobilenet_v1.pkl', 'rb') as f:
    data = pickle.load(f)

In [3]:
list(data.keys())

['mobilenet_encoder/conv_ds_10/depthwise/batch_normalization/moving_variance:0',
 'mobilenet_encoder/conv_ds_5/pointwise/batch_normalization/gamma:0',
 'mobilenet_encoder/conv_1/weights:0',
 'mobilenet_encoder/conv_ds_9/pointwise/batch_normalization/gamma:0',
 'mobilenet_encoder/conv_ds_14/depthwise/weights:0',
 'mobilenet_encoder/conv_ds_6/depthwise/weights:0',
 'mobilenet_encoder/conv_ds_11/depthwise/batch_normalization/moving_variance:0',
 'mobilenet_encoder/conv_ds_5/pointwise/weights:0',
 'mobilenet_encoder/conv_ds_4/depthwise/weights:0',
 'mobilenet_encoder/conv_ds_10/pointwise/weights:0',
 'mobilenet_encoder/conv_1/batch_normalization/moving_variance:0',
 'mobilenet_encoder/conv_ds_5/depthwise/batch_normalization/moving_variance:0',
 'mobilenet_encoder/conv_ds_13/pointwise/weights:0',
 'mobilenet_encoder/conv_ds_2/depthwise/weights:0',
 'mobilenet_encoder/conv_ds_10/depthwise/weights:0',
 'mobilenet_encoder/conv_ds_4/depthwise/batch_normalization/moving_variance:0',
 'mobilenet_

In [5]:
# Pretrained on TinyImageNet
# Weights for conv dw layer 1 - 3 x 3 x 32 dw + 1 x 1 x 32 x 64
print(data['mobilenet_encoder/conv_ds_2/depthwise/weights:0'].shape)
print(data['mobilenet_encoder/conv_ds_2/pointwise/weights:0'].shape)

# Weights for conv dw layer 13 - 3 x 3 x 512 dw + 1 x 1 x 512 x 1024
print(data['mobilenet_encoder/conv_ds_13/depthwise/weights:0'].shape)
print(data['mobilenet_encoder/conv_ds_13/pointwise/weights:0'].shape)

(3, 3, 32, 1)
(1, 1, 32, 64)
(3, 3, 512, 1)
(1, 1, 512, 1024)


In [6]:
# Weights for conv dw layer 1 - 3 x 3 x 32 dw + 1 x 1 x 32 x 64
print(data['mobilenet_encoder/conv_ds_2/depthwise/weights:0'][:,:,1,0])
# print(data['mobilenet_encoder/conv_ds_2/pointwise/weights:0'])

# Weights for conv dw layer 13 - 3 x 3 x 512 dw + 1 x 1 x 512 x 1024
# print(data['mobilenet_encoder/conv_ds_13/depthwise/weights:0'])
# print(data['mobilenet_encoder/conv_ds_13/pointwise/weights:0'])

[[-0.7965151 -2.9345374  0.8257375]
 [ 1.6487117 12.158534  -3.467292 ]
 [-2.9674451  1.5625546 -1.5359287]]


In [17]:
from utils import parse_args, create_experiment_dirs, calculate_flops
from model import MobileNet
from train import Train
from data_loader import DataLoader
from summarizer import Summarizer
import tensorflow as tf
from easydict import EasyDict as edict
import json
import numpy as np

In [18]:
def main():
    with open('test.json', 'r') as config_file:
        config_args_dict = json.load(config_file)
    config_args = edict(config_args_dict)

    # Create the experiment directories
    _, config_args.summary_dir, config_args.checkpoint_dir = create_experiment_dirs(config_args.experiment_dir)

    # Reset the default Tensorflow graph
    tf.reset_default_graph()

    # Tensorflow specific configuration
    config = tf.ConfigProto(allow_soft_placement=True)
    config.gpu_options.allow_growth = True
    sess = tf.Session(config=config)

    # Data loading
    data = DataLoader(config_args.batch_size, config_args.shuffle)
    print("Loading Data...")
    config_args.img_height, config_args.img_width, config_args.num_channels, \
    config_args.train_data_size, config_args.test_data_size = data.load_data()
    print("Data loaded\n\n")

    # Model creation
    print("Building the model...")
    model = MobileNet(config_args)
    print("Model is built successfully\n\n")

    # Summarizer creation
    summarizer = Summarizer(sess, config_args.summary_dir)
    # Train class
    trainer = Train(sess, model, data, summarizer)

    if config_args.to_train:
        try:
            print("Training...")
            trainer.train()
            print("Training Finished\n\n")
        except KeyboardInterrupt:
            trainer.save_model()

    if config_args.to_test:
        print("Final test!")
        nodes = trainer.test('val')
        print("Testing Finished\n\n")
        
    return nodes 

In [19]:
nodes = main()

Experiment directories created!
Loading Data...
Data loaded


Building the model...
Model is built successfully


Initializing the model...
Model initialized successfully


Loading ImageNet pretrained weights...


Testing:   0%|                                                                                   | 0/4 [00:00<?, ?it/s]

ImageNet Pretrained Weights Loaded Initially


No checkpoints available!


Final test!


Testing:  75%|████████████████████████████████████████████████████████▎                  | 3/4 [00:00<00:00,  3.06it/s]


Test results | test_loss: 2.0253577 - test_acc: 1.0
Testing Finished




In [42]:
list(nodes.keys())

['mobilenet_encoder/conv_1/Relu6:0',
 'mobilenet_encoder/conv_ds_2/depthwise/Relu6:0',
 'mobilenet_encoder/conv_ds_2/pointwise/Relu6:0',
 'mobilenet_encoder/conv_ds_3/depthwise/Relu6:0',
 'mobilenet_encoder/conv_ds_3/pointwise/Relu6:0',
 'mobilenet_encoder/conv_ds_4/depthwise/Relu6:0',
 'mobilenet_encoder/conv_ds_4/pointwise/Relu6:0',
 'mobilenet_encoder/conv_ds_5/depthwise/Relu6:0',
 'mobilenet_encoder/conv_ds_5/pointwise/Relu6:0',
 'mobilenet_encoder/conv_ds_6/depthwise/Relu6:0',
 'mobilenet_encoder/conv_ds_6/pointwise/Relu6:0',
 'mobilenet_encoder/conv_ds_7/depthwise/Relu6:0',
 'mobilenet_encoder/conv_ds_7/pointwise/Relu6:0',
 'mobilenet_encoder/conv_ds_8/depthwise/Relu6:0',
 'mobilenet_encoder/conv_ds_8/pointwise/Relu6:0',
 'mobilenet_encoder/conv_ds_9/depthwise/Relu6:0',
 'mobilenet_encoder/conv_ds_9/pointwise/Relu6:0',
 'mobilenet_encoder/conv_ds_10/depthwise/Relu6:0',
 'mobilenet_encoder/conv_ds_10/pointwise/Relu6:0',
 'mobilenet_encoder/conv_ds_11/depthwise/Relu6:0',
 'mobilene

In [22]:
y = nodes['mobilenet_encoder/conv_ds_2/pointwise/Relu6:0'][0][:,:,0]

(1, 112, 112, 64)

In [76]:
for i in range(1,64):
    y = np.concatenate((y, nodes['mobilenet_encoder/conv_ds_2/pointwise/Relu6:0'][0][:,:,i]), axis=None)

In [78]:
np.savetxt('activations_conv_pw32.csv', y, delimiter=',')

In [31]:
x = data['mobilenet_encoder/conv_ds_2/pointwise/weights:0'][:,:,:,0][0][0]

In [32]:
for i in range(1,64):
    x = np.concatenate((x, data['mobilenet_encoder/conv_ds_2/pointwise/weights:0'][:,:,:,i][0][0]), axis=None)

In [35]:
np.savetxt('weights_conv_pw32.csv', x, delimiter=',')

In [22]:
z = nodes['mobilenet_encoder/conv_ds_2/depthwise/Relu6:0'][0][:,:,0]
z.max()

0.0

In [23]:
for i in range(1,32):
    z = np.concatenate((z, nodes['mobilenet_encoder/conv_ds_2/depthwise/Relu6:0'][0][:,:,i]), axis=None)
z.shape

(401408,)

In [24]:
np.savetxt('activations_conv_dw32.csv', z, delimiter=',')

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [13]:
x = data['mobilenet_encoder/conv_ds_2/depthwise/weights:0'][:,:,0,0]
for i in range(1,32):
    x = np.concatenate((x, data['mobilenet_encoder/conv_ds_2/depthwise/weights:0'][:,:,i,0]), axis=None)

In [14]:
np.savetxt('weights_conv_dw32.csv', x, delimiter=',')

In [25]:
y = nodes['mobilenet_encoder/conv_1/Relu6:0'][0][:,:,0]
for i in range(1,32):
    y = np.concatenate((y, nodes['mobilenet_encoder/conv_1/Relu6:0'][0][:,:,i]), axis=None)
y.shape

(401408,)

In [26]:
np.savetxt('activations_conv1.csv', y, delimiter=',')

In [31]:
unpad_data = nodes['mobilenet_encoder/conv_1/Relu6:0'][0]
print(unpad_data.shape)
pad_data = np.zeros((114, 114, 32))
x_offset = 1
y_offset = 1
pad_data[x_offset:unpad_data.shape[0]+x_offset,y_offset:unpad_data.shape[1]+y_offset,] = unpad_data
print(pad_data.shape)

(112, 112, 32)
(114, 114, 32)


In [36]:
y = pad_data[:,:,0]
for i in range(1,32):
    y = np.concatenate((y, pad_data[:,:,i]), axis=None)
y.shape

(415872,)

In [37]:
np.savetxt('activations_conv1_pad.csv', y, delimiter=',')

In [1]:
import numpy as np
standard_conv_dummy = np.random.uniform(low=-1, high=2, size=(114,114,32))

In [3]:
y = standard_conv_dummy[:,:,0]
for i in range(1,32):
    y = np.concatenate((y, standard_conv_dummy[:,:,i]), axis=None)
y.shape

(415872,)

In [4]:
np.savetxt('standard_conv_dummy.csv', y, delimiter=',')

In [6]:
standard_dummy_wgt = np.random.uniform(low=-1, high=2, size=(3,3,32,64))

y = standard_dummy_wgt[:,:,:,0]
for i in range(1,64):
    y = np.concatenate((y, standard_dummy_wgt[:,:,:,i]), axis=None)
y.shape

(18432,)

In [7]:
np.savetxt('standard_conv_dummy_weights.csv', y, delimiter=',')

In [9]:
import numpy as np
x = data['mobilenet_encoder/conv_ds_6/pointwise/weights:0'][:,:,:,0][0][0]
for i in range(1,256):
    x = np.concatenate((x, data['mobilenet_encoder/conv_ds_6/pointwise/weights:0'][:,:,:,i][0][0]), axis=None)
x.shape

(65536,)

In [10]:
np.savetxt('weights_conv_pw256.csv', x, delimiter=',')

In [24]:
x = data['mobilenet_encoder/conv_ds_6/depthwise/weights:0'][:,:,0,0]
for i in range(1,256):
    x = np.concatenate((x, data['mobilenet_encoder/conv_ds_6/depthwise/weights:0'][:,:,i,0]), axis=None)
x.shape

(2304,)

In [25]:
np.savetxt('weights_conv_dw256.csv', x, delimiter=',')

In [39]:
y = nodes['mobilenet_encoder/conv_ds_6/depthwise/Relu6:0'][0][:,:,0]
for i in range(1,256):
    y = np.concatenate((y, nodes['mobilenet_encoder/conv_ds_6/pointwise/Relu6:0'][0][:,:,i]), axis=None)
y.shape

(200704,)

In [40]:
np.savetxt('activations_conv_dw256.csv', y, delimiter=',')

In [41]:
unpad_data = nodes['mobilenet_encoder/conv_ds_5/pointwise/Relu6:0'][0]
print(unpad_data.shape)
pad_data = np.zeros((30, 30, 256))
x_offset = 1
y_offset = 1
pad_data[x_offset:unpad_data.shape[0]+x_offset,y_offset:unpad_data.shape[1]+y_offset,] = unpad_data
print(pad_data.shape)

(28, 28, 256)
(30, 30, 256)


In [42]:
y = pad_data[:,:,0]
for i in range(1,256):
    y = np.concatenate((y, pad_data[:,:,i]), axis=None)
y.shape

(230400,)

In [43]:
np.savetxt('activations_conv256_pad.csv', y, delimiter=',')